In [5]:
import chess
import numpy as np
import polars as pl
from tqdm import tqdm, trange
from tinygrad import Tensor, nn, TinyJit


In [2]:
def fen_to_vec(fen: str) -> np.ndarray:
    vector = np.zeros(64 * 12, dtype=np.uint8)
    piece_types = [
        (chess.PAWN, chess.WHITE),
        (chess.KNIGHT, chess.WHITE),
        (chess.BISHOP, chess.WHITE),
        (chess.ROOK, chess.WHITE),
        (chess.QUEEN, chess.WHITE),
        (chess.KING, chess.WHITE),
        (chess.PAWN, chess.BLACK),
        (chess.KNIGHT, chess.BLACK),
        (chess.BISHOP, chess.BLACK),
        (chess.ROOK, chess.BLACK),
        (chess.QUEEN, chess.BLACK),
        (chess.KING, chess.BLACK),
    ]

    for i, (ptype, color) in enumerate(piece_types):
        for square in chess.Board(fen).pieces(ptype, color):
            vector[i * 64 + square] = 1

    return vector

def parse_eval(e: str) -> int:
    sign = -1 if e.startswith("-") else 1
    return int(e.strip("-").strip("+") ) * sign

In [6]:
print("Loading data")
df = pl.read_csv("./data.csv")
df = df.filter(pl.col("Evaluation").str.starts_with("#").not_())
df = df.sample(150_000)
train_df, test_df = df[:100_000], df[100_000:]
print("Loaded data")
x_train = [fen_to_vec(fen) for fen in tqdm(train_df["FEN"], desc="Processing training FENs")]
x_test = [fen_to_vec(fen) for fen in tqdm(test_df["FEN"], desc="Processing testing FENs")]
y_train = [parse_eval(e) for e in tqdm(train_df["Evaluation"], desc="Parsing training evals")]
y_test = [parse_eval(e) for e in tqdm(test_df["Evaluation"], desc="Parsing testing evals")]

Loading data
Loaded data




Processing training FENs:   0%|          | 0/100000 [00:00<?, ?it/s]

Processing training FENs:   0%|          | 294/100000 [00:00<00:33, 2938.50it/s]

Processing training FENs:   1%|          | 588/100000 [00:00<00:34, 2923.35it/s]

Processing training FENs:   1%|          | 886/100000 [00:00<00:33, 2947.18it/s]

Processing training FENs:   1%|          | 1181/100000 [00:00<00:33, 2931.58it/s]

Processing training FENs:   1%|▏         | 1483/100000 [00:00<00:33, 2961.53it/s]

Processing training FENs:   2%|▏         | 1780/100000 [00:00<00:33, 2942.51it/s]

Processing training FENs:   2%|▏         | 2075/100000 [00:00<00:33, 2936.71it/s]

Processing training FENs:   2%|▏         | 2369/100000 [00:00<00:33, 2918.89it/s]

Processing training FENs:   3%|▎         | 2661/100000 [00:00<00:33, 2888.45it/s]

Processing training FENs:   3%|▎         | 2950/100000 [00:01<00:33, 2887.53it/s]

Processing training FENs:   3%|▎         | 3239/100000 [00:01<00:33, 2853.66it/s]

Processing trainin

In [14]:
x_train = Tensor(x_train)
y_train = Tensor(y_train)
x_test = Tensor(x_test)
y_test = Tensor(y_test)

RuntimeError: can't create Tensor from <Tensor <UOp METAL (100000, 768) float (<Ops.COPY: 7>, None)> on METAL with grad None> with type <class 'tinygrad.tensor.Tensor'>

In [15]:
class Model:
    def __init__(self):
        self.l1 = nn.Linear(64*12, 512)
        self.l2 = nn.Linear(512, 256)
        self.l3 = nn.Linear(256, 64)
        self.l4 = nn.Linear(64, 1)

    def __call__(self, x: Tensor) -> Tensor:
        x = self.l1(x).relu()
        x = self.l2(x).relu()
        x = self.l3(x).relu()
        x = self.l4(x)

        return x

In [16]:
batch_size = 128
model = Model()
optim = nn.optim.Adam(nn.state.get_parameters(model))

@TinyJit
def step():
  Tensor.training = True
  samples = Tensor.randint(batch_size, high=x_train.shape[0])
  x, y = x_train[samples], y_train[samples]
  optim.zero_grad()
  loss = model(x).sub(y).pow(2).mean().backward()
  optim.step()
  return loss


In [17]:
for n in trange(7000):
    loss = step()
    if n % 100 == 0:
        Tensor.training = False
        acc = (model(x_test).argmax(axis=1) == y_test).mean().item()
        tqdm.write(f"step {n:4d}, loss {loss.item():.2f}, acc {acc * 100.:.2f}%")


  0%|          | 1/7000 [00:01<3:39:34,  1.88s/it]

step    0, loss 987333.31, acc 10.20%


  1%|          | 81/7000 [00:19<27:56,  4.13it/s] 


KeyboardInterrupt: 